In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [3]:
# Continuous variables
cont_cols = ['cont_1','cont_2','cont_3','cont_4','cont_5','cont_6','cont_7','cont_8','cont_9','cont_10','cont_11','cont_12',
            'cont_13','cont_14','cont_15','cont_16','cont_17','cont_18']

# Categorical variables
cat_cols = ['cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7','cat_8','cat_9','cat_10','cat_11','cat_12','cat_13',
           'cat_14','cat_15','cat_16', 'cat_17','cat_18','cat_19','cat_20', 'cat_21', 'cat_22','cat_23']

In [4]:
print "Number of categorical features:", len(cat_cols)
print 'Number of continuous features:', len(cont_cols)

Number of categorical features: 23
Number of continuous features: 18


In [5]:
train.head()

,connection_id,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23,target
0,cxcon_1,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2
1,cxcon_4,0,520,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
2,cxcon_7,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
3,cxcon_10,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
4,cxcon_13,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2


In [6]:
cols_to_use = list(set(train.columns) - set(['cat_17','target']))

In [7]:
X = pd.get_dummies(train[cols_to_use])

MemoryError: 

In [ ]:
y = train.target

In [ ]:
X_test = pd.get_dummies(test[cols_to_use])

In [ ]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

In [ ]:
def multAcc(pred, dtrain):
    label = dtrain.get_label()
    acc = accuracy_score(label, pred)
    return 'maccuracy', acc

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.10, random_state = 47)

In [ ]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dvalid = xgb.DMatrix(data=X_valid, label=y_valid)
dtest = xgb.DMatrix(X_test)

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
params = {}
params['objective'] = 'multi:softmax'
params['eta'] = 0.01
params['max_depth'] = 15
params['silent'] = 0
params['max_bin'] = 256
params['max_leaf_nodes'] = 2**13
params['num_class'] = 3
params['colsample_bytree'] = 1.0

In [ ]:
res = xgb.cv(params=params, dtrain=dtrain, num_boost_round=300, nfold=3, feval=multAcc, seed=0, callbacks=[xgb.callback.print_evaluation(show_stdv=False), xgb.callback.early_stop(10)])

In [ ]:
model = xgb.train(params, dtrain, 300, watchlist, maximize=True, verbose_eval=10, early_stopping_rounds=50, feval=multAcc)

In [ ]:
pred = model.predict(dtest)

In [ ]:
subm = pd.DataFrame()

In [ ]:
subm['connection_id'] = test.connection_id
subm['target'] = pred.astype('int')
subm.to_csv('submit.csv',index=False)

In [ ]:
model.

In [ ]:
#  [4]	train-maccuracy:0.782407	valid-maccuracy:0.781643